Generating landslide satellite images from Google Earth Engine and also elevation/slope bands thru projection with Nasa Dem

In [1]:
# We import earth engine, authenticate and initialize
import ee
import geemap
import geedim as gd
# ee.Authenticate()
ee.Initialize()
geemap.ee_initialize()
gd.Initialize()

In [2]:
PATH_INPUT_GEOJSON = 'data_train.geojson'
PATH_OUTPUT_ALL_IMAGES = "./generated_images/images_train_data/"

In [3]:
# The usefull dataset of images and dem models we will use
sentinel_2_collection = ee.ImageCollection("COPERNICUS/S2_HARMONIZED") #https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_HARMONIZED#bands
landsat_8_collection = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR") #https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1
landsat_7_collection = ee.ImageCollection("LANDSAT/LE07/C01/T1_SR")#https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LE07_C02_T1
nasa_dem = ee.Image('NASA/NASADEM_HGT/001').select('elevation') #https://developers.google.com/earth-engine/datasets/catalog/NASA_NASADEM_HGT_001



In [4]:
import datetime
# Dates for each of the datasets available
landsat_8_range = [datetime.datetime.strptime('2013-04-01','%Y-%m-%d'), datetime.datetime.now()]
landsat_7_range = [datetime.datetime.strptime('1999-01-01','%Y-%m-%d'), datetime.datetime.now()]
sentinel_2_range = [datetime.datetime.strptime('2015-06-23','%Y-%m-%d'), datetime.datetime.now()]
nasa_dem_time_range = [datetime.datetime.strptime('2000-02-11','%Y-%m-%d'), datetime.datetime.strptime('2000-02-22','%Y-%m-%d')]



In [5]:
# Cloud masks functions from https://github.com/fitoprincipe/geetools-code-editor/wiki/Cloud-Masks
# This functions are for cloud masking in out satellite imagery
from geetools import tools, composite, cloud_mask
sentinel_2_mask_clouds_function = cloud_mask.sentinel2()
landsat_mask_clouds_function = cloud_mask.landsatSR()


In [22]:
# Load geojson with training data to generate
import geojson
from datetime import datetime,timedelta
import geedim as gd
geojson_file = open(PATH_INPUT_GEOJSON, 'r', encoding="utf8")
geojson_file_2 = open(PATH_INPUT_GEOJSON, 'r', encoding="utf8")
geo = geojson.load(geojson_file_2)
geojson = geojson.load(geojson_file)



In [7]:
# We check number of features (data points that we will be generating the image for)
len(geojson.features)

10190

In [45]:

geo.features= []
for x in geojson.features:
    properties = x.properties
    if(properties["landslide_id"]=='21572'):
        geo.features.append(x)
        break
print(geo)
sentinel_2_images(geo,bounds_meters=10000,output_path='./')


{"crs": {"properties": {"name": "urn:ogc:def:crs:OGC:1.3:CRS84"}, "type": "name"}, "features": [{"geometry": {"coordinates": [-90.500278, 14.636389], "type": "Point"}, "properties": {"event_date": "2020-05-31", "landslide": "1", "landslide_id": "21572", "latitude": "14.63638889", "longitude": "-90.50027778"}, "type": "Feature"}], "name": "data_10000_sin_duplicados", "type": "FeatureCollection"}


There is no STAC entry for: <bound method ApiFunction.importApi.<locals>.MakeBoundFunction.<locals>.<lambda> of <ee.image.Image object at 0x000001E67C2547C0>>
pre_landsdlide_test_21572.tif: |██████████| 148M/148M (raw) [100.0%] in 00:50 (eta: 00:00)


([], [])

In [43]:
def sentinel_2_images(geojson, days_to_check_for=180, bounds_meters=50,output_path=PATH_OUTPUT_ALL_IMAGES):
    error_landslides_ids_sentinel2 = []
    landslides_not_includes_in_sentinel2 = []
    for feature in geojson.features:
        geometry = feature.geometry
        coordinates = geometry.coordinates
        properties = feature.properties
        landslide_id = properties["landslide_id"]
        event_date = properties["event_date"]
        landslide_prefix =  "" if properties["landslide"]=="1" else "non_"
        region = ee.Geometry.Point(coords=coordinates).buffer(bounds_meters).bounds()
        init_date = datetime.strptime(event_date,'%Y-%m-%d') - timedelta(days=days_to_check_for)
        end_date = datetime.strptime(event_date,'%Y-%m-%d') - timedelta(days=1)
        try:
            if(init_date>sentinel_2_range[0]):
                collection_region = sentinel_2_collection.filterDate(str(init_date).split(" ")[0], str(end_date).split(" ")[0]).filterBounds(region).filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 20)).map(sentinel_2_mask_clouds_function).sort('CLOUDY_PIXEL_PERCENTAGE', True)
                image_input = collection_region.first()
                
                #Calculate ndvi
                nir = image_input.select('B8')
                red = image_input.select('B4')
                blue = image_input.select('B2')
                ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
                green = image_input.select('B3')
                gndvi = nir.subtract(green).divide(nir.add(green)).rename('GNDVI')

                #Calculate Brightness
                brightness = (((red.add(green).add(blue)).divide(3)).divide(1000)).rename('brightness')

                #Calculate slope and elevation from nasa dem and adapt projection to input image
                imageProjection = ndvi.projection() 
                slope_dem = ee.Terrain.slope(nasa_dem).rename('slope')
                dem_projected = nasa_dem.reproject(imageProjection)
                slope_projected = slope_dem.reproject(imageProjection)
                
                #Merge the bands
                image = image_input.float().select('B4', 'B3', 'B2', 'B8').addBands(ndvi.float().select('NDVI'))
                image = image.addBands(slope_projected.float().select('slope'))
                image = image.addBands(dem_projected.float().select('elevation').rename('elevation'))
                image = image.addBands(gndvi.float().select('GNDVI'))
                image = image.addBands(brightness.float().select('brightness'))

                #Download the image
                image_input_comp = gd.MaskedImage.from_id(str(image.id))
                image_input_comp.ee_image = image
                image_input_comp.download('{}{}pre_landsdlide_test_{}.tif'.format(output_path, landslide_prefix,landslide_id), region=region, scale=10,
                                                crs='EPSG:4326',
                                                overwrite=True
                                                
                            )
            else:
                landslides_not_includes_in_sentinel2.append(landslide_id)
                print("f")
        except Exception as e: 
            print(e)
            error_landslides_ids_sentinel2.append(landslide_id)
            landslides_not_includes_in_sentinel2.append(landslide_id)
            pass
    return landslides_not_includes_in_sentinel2, error_landslides_ids_sentinel2

In [23]:
def landsat_7_images(geojson, days_to_check_for=180, bounds_meters=150,output_path=PATH_OUTPUT_ALL_IMAGES):
    error_landslides_ids_landsat7 = []
    landslides_not_includes_in_landsat7 = []
    for feature in geojson.features:
        geometry = feature.geometry
        coordinates = geometry.coordinates
        properties = feature.properties
        landslide_id = properties["landslide_id"]
        event_date = properties["event_date"]
        landslide_prefix =  "" if properties["landslide"]=="1" else "non_"
        region = ee.Geometry.Point(coords=coordinates).buffer(bounds_meters).bounds()
        init_date = datetime.strptime(event_date,'%Y-%m-%d') - timedelta(days=days_to_check_for)
        end_date = datetime.strptime(event_date,'%Y-%m-%d') - timedelta(days=1)
        try:
            if(init_date>landsat_7_range[0]):
                collection_region = landsat_7_collection.filterDate(str(init_date).split(" ")[0], str(end_date).split(" ")[0]).filterBounds(region).filter(ee.Filter.lte('CLOUD_COVER', 20)).map(landsat_mask_clouds_function).sort('CLOUD_COVER', True)
                image_input = collection_region.first()
                
                #Calculate ndvi
                nir = image_input.select('B4')
                red = image_input.select('B3')
                blue = image_input.select('B1')
                ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
                green = image_input.select('B2')
                gndvi = nir.subtract(green).divide(nir.add(green)).rename('GNDVI')

                #Calculate Brightness
                brightness = (((red.add(green).add(blue)).divide(3)).divide(1000)).rename('brightness')

                #Calculate slope and elevation from nasa dem  and adapt projection to input image
                imageProjection = ndvi.projection() 
                slope_dem = ee.Terrain.slope(nasa_dem).rename('slope')
                dem_projected = nasa_dem.reproject(imageProjection)
                slope_projected = slope_dem.reproject(imageProjection)
                
                #Merge the bands
                image = image_input.float().select('B3', 'B2', 'B1', 'B4').addBands(ndvi.float().select('NDVI'))
                image = image.addBands(slope_projected.float().select('slope'))
                image = image.addBands(dem_projected.float().select('elevation').rename('elevation'))
                image = image.addBands(gndvi.float().select('GNDVI'))
                image = image.addBands(brightness.float().select('brightness'))
        

                #Download the image
                image_input_comp = gd.MaskedImage.from_id(str(image.id))
                image_input_comp.ee_image = image
                image_input_comp.download('{}{}pre_landsdlide_test_{}.tif'.format(path_output, landslide_prefix,landslide_id), region=region, scale=10,
                                                crs='EPSG:4326',
                                                overwrite=True )
            else:
                landslides_not_includes_in_landsat7.append(landslide_id)
        except Exception as e: 
            error_landslides_ids_landsat7.append(landslide_id)
            landslides_not_includes_in_landsat7.append(landslide_id)
            pass
    return landslides_not_includes_in_landsat7, error_landslides_ids_landsat7

In [24]:
def landsat_8_images(geojson, days_to_check_for=180, bounds_meters=150,output_path=PATH_OUTPUT_ALL_IMAGES):
    error_landslides_ids_landsat8 = []
    landslides_not_includes_in_landsat8 = []
    for feature in geojson.features:
        geometry = feature.geometry
        coordinates = geometry.coordinates
        properties = feature.properties
        landslide_id = properties["landslide_id"]
        event_date = properties["event_date"]
        landslide_prefix =  "" if properties["landslide"]=="1" else "non_"
        region = ee.Geometry.Point(coords=coordinates).buffer(bounds_meters).bounds()
        init_date = datetime.strptime(event_date,'%Y-%m-%d') - timedelta(days=days_to_check_for)
        end_date = datetime.strptime(event_date,'%Y-%m-%d') - timedelta(days=1)
        try:
            if(init_date>landsat_8_range[0]):
                collection_region = landsat_8_collection.filterDate(str(init_date).split(" ")[0], str(end_date).split(" ")[0]).filterBounds(region).filter(ee.Filter.lte('CLOUD_COVER', 20)).map(landsat_mask_clouds_function).sort('CLOUD_COVER', True)
                image_input = collection_region.first()
                
                #Calculate ndvi
                nir = image_input.select('B5')
                red = image_input.select('B4')
                blue = image_input.select('B2')
                ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
                green = image_input.select('B3')
                gndvi = nir.subtract(green).divide(nir.add(green)).rename('GNDVI')

                #Calculate Brightness
                brightness = (((red.add(green).add(blue)).divide(3)).divide(1000)).rename('brightness')

                #Calculate slope and elevation from ALOS  and adapt projection to input image
                imageProjection = ndvi.projection() 
                slope_alos = ee.Terrain.slope(nasa_dem).rename('slope')
                dem_projected = nasa_dem.reproject(imageProjection)
                slope_projected = slope_alos.reproject(imageProjection)
                
                #Merge the bands
                image = image_input.float().select('B4', 'B3', 'B2', 'B5').addBands(ndvi.float().select('NDVI'))
                image = image.addBands(slope_projected.float().select('slope'))
                image = image.addBands(dem_projected.float().select('elevation').rename('elevation'))
                image = image.addBands(gndvi.float().select('GNDVI'))
                image = image.addBands(brightness.float().select('brightness'))
        

                #Download the image
                image_input_comp = gd.MaskedImage.from_id(str(image.id))
                image_input_comp.ee_image = image
                image_input_comp.download('{}{}pre_landsdlide_test_{}.tif'.format(path_output, landslide_prefix,landslide_id), region=region, scale=10,
                                                crs='EPSG:4326',
                                                overwrite=True )
            else:
                landslides_not_includes_in_landsat8.append(landslide_id)
        except Exception as e: 
            error_landslides_ids_landsat8.append(landslide_id)
            landslides_not_includes_in_landsat8.append(landslide_id)
            pass
    return landslides_not_includes_in_landsat8, error_landslides_ids_landsat8

In [27]:
def get_satellite_available_images(geojson, days_to_check_for=180, bounds_meters=50,output_path=PATH_OUTPUT_ALL_IMAGES):
    error_landslides_ids_all = []
    landslides_not_includes_in_all = []
    for feature in geojson.features:
        geometry = feature.geometry
        coordinates = geometry.coordinates
        properties = feature.properties
        landslide_id = properties["landslide_id"]
        event_date = properties["event_date"]
        landslide_prefix =  "" if properties["landslide"]=="1" else "non_"
        region = ee.Geometry.Point(coords=coordinates).buffer(bounds_meters).bounds()
        end_date = datetime.strptime(event_date,'%Y-%m-%d') - timedelta(days=1)
        init_date = datetime.strptime(event_date,'%Y-%m-%d') - timedelta(days=days_to_check_for)
        try:
            if(init_date>sentinel_2_range[0]):
                collection_region = sentinel_2_collection.filterDate(str(init_date).split(" ")[0], str(end_date).split(" ")[0]).filterBounds(region).filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 20)).map(sentinel_2_mask_clouds_function).sort('CLOUDY_PIXEL_PERCENTAGE', True)
                image_input = collection_region.first()
                
                #Calculate ndvi
                nir = image_input.select('B8')
                red = image_input.select('B4')
                blue = image_input.select('B2')
                ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
                green = image_input.select('B3')
                gndvi = nir.subtract(green).divide(nir.add(green)).rename('GNDVI')

                #Calculate Brightness
                brightness = (((red.add(green).add(blue)).divide(3)).divide(1000)).rename('brightness')

                #Calculate slope and elevation from nasa dem  and adapt projection to input image
                imageProjection = ndvi.projection() 
                slope_dem = ee.Terrain.slope(nasa_dem).rename('slope')
                dem_projected = nasa_dem.reproject(imageProjection)
                slope_projected = slope_dem.reproject(imageProjection)
                
                #Merge the bands
                image = image_input.float().select('B4', 'B3', 'B2', 'B8').addBands(ndvi.float().select('NDVI'))
                image = image.addBands(slope_projected.float().select('slope'))
                image = image.addBands(dem_projected.float().select('elevation').rename('elevation'))
                image = image.addBands(gndvi.float().select('GNDVI'))
                image = image.addBands(brightness.float().select('brightness'))
            
                image_input_comp = gd.MaskedImage.from_id(str(image.id))
                image_input_comp.ee_image = image
                image_input_comp.download('{}{}_sentinel2_pre_landsdlide_test_{}.tif'.format(output_path,landslide_prefix,landslide_id), region=region, scale=10,
                                                crs='EPSG:4326',
                                                overwrite=True
                                                
                            )
            elif(init_date>landsat_8_range[0]):
                collection_region = landsat_8_collection.filterDate(str(init_date).split(" ")[0], str(end_date).split(" ")[0]).filterBounds(region).filter(ee.Filter.lte('CLOUD_COVER', 20)).map(landsat_mask_clouds_function).sort('CLOUD_COVER', True)
                image_input = collection_region.first()
                
                #Calculate ndvi
                nir = image_input.select('B5')
                red = image_input.select('B4')
                blue = image_input.select('B2')
                ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
                green = image_input.select('B3')
                gndvi = nir.subtract(green).divide(nir.add(green)).rename('GNDVI')

                #Calculate Brightness
                brightness = (((red.add(green).add(blue)).divide(3)).divide(1000)).rename('brightness')

                #Calculate slope and elevation from nasa dem  and adapt projection to input image
                imageProjection = ndvi.projection() 
                slope_dem = ee.Terrain.slope(nasa_dem).rename('slope')
                dem_projected = nasa_dem.reproject(imageProjection)
                slope_projected = slope_dem.reproject(imageProjection)
                
                #Merge the bands
                image = image_input.float().select('B4', 'B3', 'B2', 'B5').addBands(ndvi.float().select('NDVI'))
                image = image.addBands(slope_projected.float().select('slope'))
                image = image.addBands(dem_projected.float().select('elevation').rename('elevation'))
                image = image.addBands(gndvi.float().select('GNDVI'))
                image = image.addBands(brightness.float().select('brightness'))
            
                image_input_comp = gd.MaskedImage.from_id(str(image.id))
                image_input_comp.ee_image = image
                image_input_comp.download('{}{}_landsat8_pre_landsdlide_test_{}.tif'.format(output_path,landslide_prefix,landslide_id), region=region, scale=10,
                                                crs='EPSG:4326',
                                                overwrite=True
                                                
                            )
            elif (init_date>landsat_7_range[0]):
                collection_region = landsat_7_collection.filterDate(str(init_date).split(" ")[0], str(end_date).split(" ")[0]).filterBounds(region).filter(ee.Filter.lte('CLOUD_COVER', 20)).map(landsat_mask_clouds_function).sort('CLOUD_COVER', True)
                image_input = collection_region.first()
                
                #Calculate ndvi
                nir = image_input.select('B4')
                red = image_input.select('B3')
                blue = image_input.select('B1')
                ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
                green = image_input.select('B2')
                gndvi = nir.subtract(green).divide(nir.add(green)).rename('GNDVI')

                #Calculate Brightness
                brightness = (((red.add(green).add(blue)).divide(3)).divide(1000)).rename('brightness')

                #Calculate slope and elevation from nasa dem  and adapt projection to input image
                imageProjection = ndvi.projection() 
                slope_dem = ee.Terrain.slope(nasa_dem).rename('slope')
                dem_projected = nasa_dem.reproject(imageProjection)
                slope_projected = slope_dem.reproject(imageProjection)
                
                #Merge the bands
                image = image_input.float().select('B3', 'B2', 'B1', 'B4').addBands(ndvi.float().select('NDVI'))
                image = image.addBands(slope_projected.float().select('slope'))
                image = image.addBands(dem_projected.float().select('elevation').rename('elevation'))
                image = image.addBands(gndvi.float().select('GNDVI'))
                image = image.addBands(brightness.float().select('brightness'))
            
                image_input_comp = gd.MaskedImage.from_id(str(image.id))
                image_input_comp.ee_image = image
                image_input_comp.download('{}{}_landsat7_pre_landsdlide_test_{}.tif'.format(output_path,landslide_prefix,landslide_id), region=region, scale=10,
                                                crs='EPSG:4326',
                                                overwrite=True
                                                
                            )   
            else:
                landslides_not_includes_in_all.append(landslide_id)
        except Exception as e: 
            print(e)
            error_landslides_ids_all.append(landslide_id)
            landslides_not_includes_in_all.append(landslide_id)
            pass
    print("Process of getting images completed")
    return  landslides_not_includes_in_all, error_landslides_ids_all

In [30]:
# Function to get elevation from point in date
def get_satellite_elevation_for_point(coordinates,date, days_to_check_for=180, bounds_meters=150,):
    region = ee.Geometry.Point(coords=coordinates).buffer(50).bounds()
    end_date = datetime.strptime(event_date,'%Y-%m-%d') - timedelta(days=1)
    init_date = datetime.strptime(event_date,'%Y-%m-%d') - timedelta(days=days_to_check_for)
    try:
        if(init_date>sentinel_2_range[0]):
            collection_region = sentinel_2_collection.filterDate(str(init_date).split(" ")[0], str(end_date).split(" ")[0]).filterBounds(region).filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 20)).map(sentinel_2_mask_clouds_function).sort('CLOUDY_PIXEL_PERCENTAGE', True)
            image_input = collection_region.first()
            
            #Calculate ndvi
            nir = image_input.select('B8')
            red = image_input.select('B4')
            blue = image_input.select('B2')
            ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
            green = image_input.select('B3')
            gndvi = nir.subtract(green).divide(nir.add(green)).rename('GNDVI')


            #Calculate slope and elevation from nasa dem  and adapt projection to input image
            imageProjection = ndvi.projection() 
            slope_dem = ee.Terrain.slope(nasa_dem).rename('slope')
            dem_projected = nasa_dem.reproject(imageProjection)
            slope_projected = slope_dem.reproject(imageProjection)
            return dem_projected.float().select('elevation')
            
        elif(init_date>landsat_8_range[0]):
            collection_region = landsat_8_collection.filterDate(str(init_date).split(" ")[0], str(end_date).split(" ")[0]).filterBounds(region).filter(ee.Filter.lte('CLOUD_COVER', 20)).map(landsat_mask_clouds_function).sort('CLOUD_COVER', True)
            image_input = collection_region.first()
            
            #Calculate ndvi
            nir = image_input.select('B5')
            red = image_input.select('B4')
            blue = image_input.select('B2')
            ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
            green = image_input.select('B3')
            gndvi = nir.subtract(green).divide(nir.add(green)).rename('GNDVI')
            #Calculate slope and elevation from nasa dem  and adapt projection to input image
            imageProjection = ndvi.projection() 
            slope_dem = ee.Terrain.slope(nasa_dem).rename('slope')
            dem_projected = nasa_dem.reproject(imageProjection)
            slope_projected = slope_dem.reproject(imageProjection) 
            return dem_projected
            
        elif (init_date>landsat_7_range[0]):
            collection_region = landsat_7_collection.filterDate(str(init_date).split(" ")[0], str(end_date).split(" ")[0]).filterBounds(region).filter(ee.Filter.lte('CLOUD_COVER', 20)).map(landsat_mask_clouds_function).sort('CLOUD_COVER', True)
            image_input = collection_region.first()
            
            #Calculate ndvi
            nir = image_input.select('B4')
            red = image_input.select('B3')
            blue = image_input.select('B1')
            ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
            green = image_input.select('B2')
            gndvi = nir.subtract(green).divide(nir.add(green)).rename('GNDVI')

            
            #Calculate slope and elevation from nasa dem  and adapt projection to input image
            imageProjection = ndvi.projection() 
            slope_dem = ee.Terrain.slope(nasa_dem).rename('slope')
            dem_projected = nasa_dem.reproject(imageProjection)
            slope_projected = slope_dem.reproject(imageProjection)
            
            return dem_projected
        else:
            return "Could not get dem projected"
    except Exception as e: 
        print(e)
        return "Could not get dem projected"


In [38]:
#Run code to get images
get_satellite_available_images(geojson)

There is no STAC entry for: <bound method ApiFunction.importApi.<locals>.MakeBoundFunction.<locals>.<lambda> of <ee.image.Image object at 0x0000014C72D57DC0>>
non__landsat7_pre_landsdlide_test_100008.tif: |██████████| 4.75k/4.75k (raw) [100.0%] in 00:01 (eta: 00:00)
non__landsat7_pre_landsdlide_test_100016.tif: |██████████| 4.75k/4.75k (raw) [100.0%] in 00:01 (eta: 00:00)


KeyboardInterrupt: 